In [51]:
import os
import glob
import numpy as np
import cv2
from skimage.io import imread
from skimage.io import imsave
from skimage.color import rgb2gray
from skimage.util import random_noise
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image



In [52]:
def AugmentImage(img, folder_name, image_prefix, N):
    
    """ This function uses the keras ImageDataGenerator for image augmentation.
    
    Arguments:
    
    img : Image that needs to be augmented
    folder_name : name of the folder where the images will be saved
    image_prefix : prefix for the saved images
    N : number of images that will be created through augmentation
    
    Returns:
    
    N augmented images with 'image_prefix' saved in 'folder_name'
    
    Raises:
    
    Error when input data type is incorrect
    
    """
    
    assert type(folder_name) == str, ('Wrong Data type', 'folder_name must be a string')
    assert type(image_prefix) == str, ('Wrong Data type', 'image_prefix must be a string')
    assert type(N) == int, ('Wrong Data type', 'N must be an integer')
    
    directory = './Multislice'+"/"+image_prefix+"/"+folder_name
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    datagen = ImageDataGenerator(rotation_range=10,
                                 shear_range=0.2,
                                 zoom_range=[0.7,1],
                                 rescale=1.0/225,
                                 horizontal_flip=True, 
                                 vertical_flip=True, 
                                 fill_mode='constant', 
                                 cval = 0.0)

    
    x = img_to_array(img)  # this is a Numpy array with shape (64, 64, 1)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 64, 64, 1)

    i = 0
    #save the image data into each type of defect directory 
    for batch in datagen.flow(x, batch_size = 1,
                              save_to_dir = './Multislice'+'/'+image_prefix+'/'+folder_name, save_prefix = image_prefix, save_format = 'png'):
        i += 1
        if i > N:
            break  # otherwise the generator would loop indefinitely
    
    return

In [53]:
def AddNoiseImage(img, folder_name, image):
    
    #img is the augemented data you read in
    #folder_name: for example, if it is 'Mo', it means the raw data is from folder 'Mo', and the 
    #             image with noise will be saved under 'Mo/Mo_augmented/Mo_noise'
    #image: the filename of augmented image, after add noise to this image, a new image namedby
    #       '"noise type"+image ' will be saved
    
    assert type(folder_name) == str, ('Wrong Data type', 'folder_name must be a string')
    assert type(image) == str, ('Wrong Data type', 'image must be a string')
    
    directory = './Multislice' +'/'+ folder_name+'/'+folder_name+'_Augmented'+'/'+folder_name+'_noise'
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    img_grey=rgb2gray(img)
    img_gauss=random_noise(img_grey,mode='gaussian',mean=0.,var=0.01)
    img_sp=random_noise(img_grey,mode='s&p',salt_vs_pepper=0.5)
    img_poisson=random_noise(img_grey,mode='poisson')
    
    imsave(directory+'/'+'gauss_'+image,img_gauss)
    imsave(directory+'/'+'s&p_'+image,img_sp)
    imsave(directory+'/'+'poisson'+image,img_poisson)
    
    return 

In [54]:
# the path of the folder includes different type of defects (Multislice)
path = "./Multislice"
folders=os.listdir(path) 
# folders is the list of subfolders' names (types of defects' names:Mo,Sw,Vs2,Vw,W2S2,Ws)

for folder in folders:  #traversing all subfolders(types of defects) in MULTISLICE
    if '.' in folder:   #make sure it will not traverse file like '.DS_store'
        continue
        
    path = "./Multislice"+"/"+folder
    files=os.listdir(path)
    #files is the list of all raw_image under each type of defect
    for file in files: #traversing all raw_image in the folder of defect
        if file.endswith(".png"):
            img=Image.open(path+'/'+file).convert('L') # loading the image
            AugmentImage(img, folder+'_Augmented', folder, 200) # Augmenting using the function above
    
    
    noisy_path = "./Multislice"+"/"+folder+'/'+folder+'_Augmented'  
    # the path of data_Augmented folder
    image=os.listdir(noisy_path)
    # image is the list all the augmented data
    for i in image:
        if i.endswith(".png"):
            img=imread(noisy_path+'/'+i)
            AddNoiseImage(img, folder, i)   #for each augmented image generate 3 image with gaussion,s&p,poisson noise
            
        else: 
            pass
        
       

/Applications/miniconda3/envs/keras/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
